## Run AI reviewer for application


In [25]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load variables from .env

from llm import create_client
from review import extract_pdf_text, perform_review
import pandas as pd

# Create a real client
api_key = os.getenv("OPENAI_API_KEY")
client, model = create_client("gpt-4o-2024-05-13")

app_paths = ["001", "002"]
path = "docs/application/"
output_path = "docs/review/"

results = []
# feedback_results= []
for application in app_paths:
    print(f"Review for {application} was started...")
    # Output format
    result = {}

    # Load application and related document
    file_path = path + application
    application_text = extract_pdf_text(file_path + "/application.pdf")
    contract_text = extract_pdf_text(file_path + "/contract.pdf")

    # Call the function
    review = perform_review(
        application_text=application_text,
        contract_text=contract_text,
        model=model,
        client=client,
    )

    # Convert the dictionary to a string representation
    result["review_result"] = review
    table = pd.DataFrame.from_dict(review['comments'])
    review_text = table.to_markdown(index=False)

    # # Call the function
    # feedback = provide_feedback(
    #     application_text=app_docs,
    #     review_text=review_text,
    #     model=model,
    #     client=client,
    # )
    # result["feedback_result"] = feedback

    output_file = f"{output_path}{application}.md"
    # Save as a properly formatted Markdown file
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"# Review Result for Application: {application}\n\n")
        
        f.write("## Review Result\n\n")
        f.write(table.to_markdown())
        f.write("```\n\n")

        # f.write("## Feedback\n\n")
        # f.write(feedback + "\n")

    print(f"Review for {application} was finished!")

# Print the result
print("Review was completed")

Using OpenAI API with model gpt-4o-2024-05-13.
Review for 001 was started...
Review for 001 was finished!
Review for 002 was started...
Review for 002 was finished!
Review was completed


In [22]:
pd.DataFrame.from_dict(review['comments'])

,Decision,LC description,Contract description
0,Not Mentioned,Percentage Credit Amount Tolerance: 10%,Not mentioned
1,Not Mentioned,Confirmation Instructions: Confirm,Not mentioned
2,Not Mentioned,Confirmation charges to be borne by Applicant,Not mentioned
3,Contradicts,Latest Shipment Date: 2025/02/28,Shipment Period: As per bill(s) of lading date...
4,Not Mentioned,Documents must be presented to the Negotiating...,Not mentioned
5,Not Mentioned,All banking charges outside Japan are for the ...,Not mentioned


In [17]:
review['comments']

[{'Decision': 'Not Mentioned',
  'LC description': 'Percentage Credit Amount Tolerance: 10%',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Not Mentioned',
  'LC description': 'Confirmation Instructions: Confirm',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Not Mentioned',
  'LC description': 'Confirmation charges to be borne by Applicant',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Contradicts',
  'LC description': 'Latest Shipment Date: 2025/02/28',
  'Contract description': 'Shipment Period: As per bill(s) of lading dated or to be dated within January 1st 2025 to February 28th 2025'},
 {'Decision': 'Not Mentioned',
  'LC description': 'Documents must be presented to the Negotiating Bank within 8 days after the date of shipment.',
  'Contract description': 'Not mentioned'},
 {'Decision': 'Not Mentioned',
  'LC description': 'All banking charges outside Japan are for the account of Beneficiary.',
  'Contract description': 'Not mentioned'}]

In [26]:
part = """2. Applicant & Beneficiary Details
●
Name & Address of the Applicant:
XYZ Imports Ltd., 5678 Shibuya Street, Tokyo 150-0001, Japan
●
Name & Address of the Beneficiary:
ABC Trading Co., 1234 Market Street, Suite 500, New York, NY 10001, USA
●
Currency & Amount of Credit:
○
(Figures): USD 1,600,00
○
(Words): USD one million and six hundred thousand"""

In [27]:
from llm import get_response_from_llm

it_reviewer_system_prompt = (
    "You are an AI reviewer in the international trading company responsible for evaluating Letter of Credit (L/C) applications. "
    "Your mission is to assess if all contents described in L/C application are align with the contract. "
    "Be critical and cautious in your decision-making process. Your evaluations should be structured, concise, and accurate."
)

contract_text = extract_pdf_text(file_path + "/contract.pdf")

base_prompt = f"""
Below paragraph is from L/C application. Please carefully read the content and advice if any discrepancy exists with given contract.

PARAGRAPH FROM L/C APPLICATION:
{part}

PROVIDED CONTRACT:
{contract_text}
"""


llm_review, msg_history = get_response_from_llm(
    base_prompt,
    model=model,
    client=client,
    system_message=it_reviewer_system_prompt,
    print_debug=False,
)

print(llm_review)

Upon reviewing both the Letter of Credit (L/C) application and the provided sales contract, I have identified the following discrepancies and considerations:

### Discrepancies:
1. **Address of the Applicant (Buyer):**
   - **L/C Application:** 5678 Shibuya Street, Tokyo 150-0001, Japan
   - **Sales Contract:** 5678 Shibuya Street, Tokyo 150-0002, Japan
   - **Discrepancy:** The postal code for the address of the applicant differs between the L/C application (150-0001) and the sales contract (150-0002).

2. **Currency & Amount of Credit:**
   - **L/C Application:**
     - (Figures): USD 1,600,00
     - (Words): USD one million and six hundred thousand
   - **Sales Contract:**
     - Total Contract Value: USD 1,600,000
   - **Discrepancy:** There is a typographical error in the figures mentioned in the L/C application. It states “USD 1,600,00” instead of “USD 1,600,000”.

### Considerations:
1. **Names of Applicant and Beneficiary:**
   - The names and addresses of both the Applicant (X

## Simple Comparison


In [40]:
contract_text = extract_pdf_text(file_path + "/contract.pdf")
application_text = extract_pdf_text(file_path + "/application.pdf")

base_prompt = f"""
Below is a L/C application. Please carefully read the content and advice if any discrepancy exists with given contract.
Please only provide identified discrepancy in your response.
If you do not find any discrepancy, just mention no discrepancy.

L/C APPLICATION:
{application_text}

PROVIDED CONTRACT:
{contract_text}
"""

llm_review, msg_history = get_response_from_llm(
    base_prompt,
    model=model,
    client=client,
    system_message=it_reviewer_system_prompt,
    print_debug=False,
)

print(llm_review)

**Identified Discrepancies:**

1. **Applicant Address:**
   - L/C Application: 5678 Shibuya Street, Tokyo 150-0001, Japan
   - Contract: 5678 Shibuya Street, Tokyo 150-0002, Japan

2. **Quality of Soybeans:**
   - L/C Application: USDA No.2 Standard
   - Contract: USDA No.1 Standard

3. **Price Terms:**
   - L/C Application: CFR
   - Contract: CIF (Cost, Insurance, and Freight)

4. **Insurance Policy Coverage:**
   - L/C Application: Full Invoice Value + 5%
   - Contract: Full Invoice Value + 10%

No other discrepancies were noted between the L/C application and the contract.


In [28]:
file_path

'docs/application/002'

## Break down and comparison


In [41]:
import re
import json

base_prompt = """
Please split following document into multiple parts by content.
*Please just split the original text without changing any words even if original text contains typo or mistake.
*Please split without overlapping
*Please provide output in JSON.

Args: 
    application text (str): A texts from document 

Returns:
    dict: A list of splitted text.

    output format should be as below:
    * Please only provide JSON output as string starting from { and ending with }

    {
        "output": ["list of splitted texts", "list of splitted texts", "list of splitted texts"...]
        ...
    }

"""

application_text = extract_pdf_text(file_path + "/application.pdf")
base_prompt += f"Document: {application_text}"

llm_review, msg_history = get_response_from_llm(
    base_prompt,
    model=model,
    client=client,
    system_message=it_reviewer_system_prompt,
    print_debug=False,
)

try:
    json_match = re.search(r"{.*}", llm_review, re.DOTALL)
    if json_match:
        cleaned_review = json_match.group(0)
        review = json.loads(cleaned_review)
    else:
        raise ValueError("No JSON found in response")
except json.JSONDecodeError as e:
    print(f"JSON decoding failed: {e}")
    print(f"LLM response: {llm_review}")
    review = None

print(review)

{'output': ['APPLICATION FOR IRREVOCABLE DOCUMENTARY LETTER OF CREDIT \nDate: January 5th 2025 \nTo:\u200b\n The MUFG Bank, Ltd. \nDear Sirs, \nPursuant to the General Undertaking for Letters of Credit dated January 15th 2020, we \nhereby request you to issue an Irrevocable Documentary Letter of Credit in accordance with \nthe instructions marked below. \nWe are authorized to debit our account No. 12345678 towards charges for this transaction.', '1. Credit Details \n●\u200b Type of LC: Not Transferable \n●\u200b L/C No. (For Bank Use): [Insert L/C Number] \n●\u200b Date of Issue (For Bank Use): [Insert Date] \n●\u200b UCP/URR Latest Version: UCP 600 & URR 725 \n●\u200b Date of Expiry: 2025/02/28 \n●\u200b Place of Expiry: Tokyo, Japan', '2. Applicant & Beneficiary Details \n●\u200b Name & Address of the Applicant:\u200b\n XYZ Imports Ltd., 5678 Shibuya Street, Tokyo 150-0001, Japan\u200b\n\n●\u200b Name & Address of the Beneficiary:\u200b\nABC Trading Co., 1234 Market Street, Suite 500

In [42]:
contract_text = extract_pdf_text(file_path + "/contract.pdf")
result = ""
for text in review["output"]:
    base_prompt = f"""
    Below paragraph is extracted from a L/C application. Please carefully read the content and advice if any discrepancy exists with given contract.
    Please only provide identified discrepancy in your response.
    If you do not find any discrepancy, just mention no discrepancy.

    EXTRACTED PARAGRAPH FROM L/C APPLICATION:
    {text}

    PROVIDED CONTRACT:
    {contract_text}
    """

    llm_review, msg_history = get_response_from_llm(
        base_prompt,
        model=model,
        client=client,
        system_message=it_reviewer_system_prompt,
        print_debug=False,
    )

    result += llm_review

print(result)
    

### Identified Discrepancy:

- **Account Number for Debit**: The L/C application mentions debiting account No. 12345678, while the provided contract specifies the designated bank account number for payments as 123456789. There is a discrepancy in the account number for transactions.

No other discrepancies are identified between the L/C application and the provided contract.Discrepancy identified:

- The Date of Issue field in the L/C application is marked as "[Insert Date]" for bank use. According to the contract, the L/C needs to be issued in a specific timeframe to match the shipment and delivery schedule. The contract specifies a shipment period of January 1st, 2025 to February 28th, 2025, so the L/C should ideally be issued before this period to ensure proper coordination. The issuance date should be clarified and specified in the L/C application to avoid any misalignment with the contract's shipment period.1. **Address of the Buyer (XYZ Imports Ltd.)**:
   - **L/C Application**: 